In [1]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

import os
import pandas as pd

#### Fetch credentials

In [2]:
d = {}

cwd = os.getcwd()
file_path = cwd + '/../mysql/creds.txt'

with open(file_path, 'r') as f:
    for l in f.readlines(): d[l.split()[0]] = l.split()[1]

### Create project database

In [3]:
conn = mysql.connector.connect(user=d['user'], password=d['pw'])
cur = conn.cursor()

DB_NAME = 'bi_project'

In [4]:
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)
try:
    cur.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cur)
        print("Database {} created successfully.".format(DB_NAME))
        conn.database = DB_NAME
    else:
        print(err)
        exit(1)

In [5]:
query = """
    drop table if exists Dim_Municipality
"""
cur.execute(query)

query = """
    create table Dim_Municipality (
        loc_id int not null,
        municipality varchar(20) not null,
        primary key(loc_id)
    )
"""
cur.execute(query)

query = f"""
    load data infile '{cwd}//src//municipalities//loc_id_municipality.csv'
    into table Dim_Municipality
    fields terminated by ',' 
    enclosed by '"'
    lines terminated by '\r\n'
    ignore 1 rows
    """
cur.execute(query) 

In [6]:
query = """
    select *
    from Dim_Municipality
    limit 3
"""
pd.read_sql(query, conn)

,loc_id,municipality
0,0,Grimstad
1,1,Arendal
2,2,Birkenes


In [ ]:
query = """
    drop table if exists Municipality_Nearest_Station
"""
cur.execute(query)

query = """
    create table Municipality_Nearest_Station (
        municipality varchar(20) not null,
        nearest_station varchar(20) not null,
        primary key(municipality)
    )
"""
cur.execute(query)

query = f"""
    load data infile '{cwd}//src//municipalities//municipality_nearest_station.csv'
    into table Municipality_Nearest_Station
    fields terminated by ',' 
    enclosed by '"'
    lines terminated by '\r\n'
    ignore 1 rows
    """
cur.execute(query) 

In [ ]:
query = """
    select *
    from Municipality_Nearest_Station
    limit 3
"""
pd.read_sql(query, conn)

Gather all elspotprices

In [ ]:
query = """
    drop table if exists Elspot_Prices_Gathered
"""
cur.execute(query)

query = """
    create temporary table Elspot_Prices_Gathered (
        time datetime not null,
        sys varchar(10),
        se1 varchar(10),
        se2 varchar(10),
        se3 varchar(10),
        se4 varchar(10),
        f1 varchar(10),
        dk1 varchar(10),
        dk2 varchar(10),
        oslo varchar(10),
        kristiansand varchar(10),
        bergen varchar(10),
        molde varchar(10),
        trondheim varchar(10),
        tomso varchar(10),
        ee varchar(10),
        lv varchar(10),
        lt varchar(10),
        primary key(time)
    )
"""
cur.execute(query)

In [ ]:
folder = f'{cwd}//src//elspot_prices//'
for file in os.listdir(folder):
    if file.split('.')[-1] != 'csv': continue
    file_path = os.path.join(folder, file)

    query = f"""
        load data infile '{file_path}'
        ignore into table Elspot_Prices_Gathered
        fields terminated by ',' 
        enclosed by '"'
        lines terminated by '\r\n'
        ignore 1 rows
    """
    cur.execute(query) 

In [ ]:
query = """
    select *
    from Elspot_Prices_Gathered
    limit 3
"""
pd.read_sql(query, conn)

And store those for kristiansand, which is the correct price for this part of norway.

In [ ]:
query = """
    drop table if exists Dim_Elspot
"""
cur.execute(query)

query = """
    create table Dim_Elspot
    as (
        select
            time, 
            replace(kristiansand, ',', '.') as price
        from Elspot_Prices_Gathered
    )
"""
cur.execute(query)

In [ ]:
query = """
    select count(*)
    from Dim_Elspot
    limit 5
"""
pd.read_sql(query, conn)

In [ ]:
query = """
    drop table if exists Belastning_Gathered
"""
cur.execute(query)

query = """
    create temporary table Belastning_Gathered (
        a varchar(10),
        time datetime not null,
        consumption_kvah varchar(20),
        upper_tol_kvah varchar(10),
        upper_tol_p20 varchar(10),
        loc_id int not null,
        primary key(time, loc_id)
    )
"""
cur.execute(query)

In [ ]:
folder = f'{cwd}//src//belastning_nettstasjon//'
for file in os.listdir(folder):
    if file.split('.')[-1] != 'csv': continue
    file_path = os.path.join(folder, file)

    query = f"""
        load data infile '{file_path}'
        into table Belastning_Gathered
        fields terminated by ',' 
        enclosed by '"'
        lines terminated by '\r\n'
        ignore 1 rows
    """
    cur.execute(query) 

In [ ]:
query = """
    drop table if exists Dim_Belastning
"""
cur.execute(query)

query = """
    create table Dim_Belastning as (
        select 
            time,
            loc_id,
            consumption_kvah,
            upper_tol_kvah
        from Belastning_Gathered
    )
"""
cur.execute(query)

query = """
    alter table Dim_Belastning
    add primary key(time, loc_id)
"""
cur.execute(query)

In [ ]:
query = """
    select 
        loc_id,
        count(*) as n_entries
    from Dim_Belastning
    group by loc_id
    order by loc_id
"""
pd.read_sql(query, conn).T

In [ ]:
query = """
    select *
    from Dim_Belastning
    where time like '%2019-12%'
    and loc_id = 2
    limit 10
"""
pd.read_sql(query, conn)

In [ ]:
query = """
    drop table if exists Dim_Oilspot
"""
cur.execute(query)

query = """
    create table Dim_Oilspot(
        day date not null,
        price varchar(10),
        primary key(day)
    )
"""
cur.execute(query)

In [ ]:
query = f"""
    load data infile '{cwd}//src//oilspot_prices//DCOILBRENTEU.csv'
    ignore into table Dim_Oilspot
    fields terminated by ',' 
    enclosed by '"'
    lines terminated by '\n'
    ignore 1 rows
"""
cur.execute(query) 

In [ ]:
query = """
    select *
    from Dim_Oilspot
    where day like '%2020-01%'
    limit 3
"""
pd.read_sql(query, conn)

Import weather reports either as `.json` files or run the last bit of preprocessing in `frost_api_to_json.ipynb` to convert them to `.csv` files instead.

The following cell will (on my computer at least) demand of you 33 hours of waiting. The mentioned preprocessing takes about 40 minutes. Better do that and skip this cell.

#### Alternative 1: Load `.json` files into weather table

In [ ]:
# from time import time
# 
# query = """
#     drop table if exists jsonTable
# """
# cur.execute(query)
# 
# query = """
#     create temporary table jsonTable ( 
#         jsonFile json
#     )
# """
# cur.execute(query)
# 
# # For every .json file in folder...
# folder = 'D://_bi//src//weather_jsons//'
# for file in os.listdir(folder):
#     if file.split('.')[-1] != 'json': continue
#     file_path = os.path.join(folder, file)
# 
#     # Save the file content in variable @jsonData
#     query = f"""
#         load data infile '{file_path}'
#         into table jsonTable
#         fields terminated by '\0' escaped by ''
#         lines terminated by '\0'
#         (@jsonData)
#     """
#     cur.execute(query)
#     
#     # The data contains a time series of length n
#     query = """
#         select json_length(json_extract(@jsonData, '$.data')) 
#     """
#     cur.execute(query)
#     n = cur.fetchall()[0][0]
#     
#     tic = time()
#     print(f'Rows of {file} loaded into table'.rjust(20), 'Mins elapsed'.rjust(20), 'Estimated minutes left'.rjust(30))
# 
#     # For every i = (0, 1,..., n-1) in @jsonData's data[i] array
#     # Store the ith timestamp, weather_station id, and temperature measurement
#     for i in range(n):
#         query = f"""
#             insert into Dim_Weather (time, weather_station, temp)
# 
#             with a as(
#                 select json_extract(@jsonData, '$.data[{i}].sourceId') as ws
#             ), b as (
#                 select substring(a.ws, 2, char_length(a.ws) - 4) as ws_trimmed
#                 from a
#             ), c as (
#                 select json_extract(@jsonData, '$.data[{i}].referenceTime') as rt
#             ), d as (
#                 select substring(c.rt, 2, char_length(c.rt) - 7) as rt_trimmed
#                 from c
#             ), e as (
#                 select replace(d.rt_trimmed, 'T', ' ') as time
#                 from d
#             ), f as (
#                 select json_extract(@jsonData, '$.data[{i}].observations[0].value') as temp
#             )
#             select
#                 e.time,
#                 b.ws_trimmed,
#                 f.temp
#             from
#                 e, b, f   
#         """
#         cur.execute(query)
#         
#         # Every 10 rows, print a status update
#         if not i % 10: 
#             toc = time()
#             print(f'{i}/{n}'.rjust(0), f'{(toc-tic)/60:.2f}'.rjust(42), f'{(n/(i+1) * (toc-tic) - (toc-tic))/60 :.2f}'.rjust(22), end='\r')
#             
#     print(f'Loaded file {file} into Dim_Weather')

#### Alternative 2: Preprocess `.json` files to `.csv` in `frost_api_to_json.ipynb` and load `.csv` files into weather table

In [ ]:
query = """
    drop table if exists Weather_Gathered
"""
cur.execute(query)

query = """
    create temporary table Weather_Gathered (
        a int,
        b int, 
        element_id varchar(30),
        c int,
        level varchar(70),
        perf varchar(2),
        d int, 
        timeoff varchar(10),
        timeres varchar(10),
        times_id varchar(2),
        unit varchar(5),
        value varchar(10),
        referencetime datetime not null,
        source_id varchar(20) not null,
        primary key(referencetime, source_id)
    )
"""
cur.execute(query)

In [ ]:
folder = f'{cwd}//src//weather_csvs//'

for file in os.listdir(folder):
    if file.split('.')[-1] != 'csv': continue
    file_path = os.path.join(folder, file)
    
    query = f"""
        load data infile '{file_path}'
        into table Weather_Gathered
        fields terminated by ',' 
        enclosed by '"'
        lines terminated by '\r\n'
        ignore 1 rows  
    """
    cur.execute(query)

In [ ]:
query = """
    select *
    from Weather_Gathered
    limit 3
"""
pd.read_sql(query, conn)

In [ ]:
query = """
    drop table if exists Dim_Weather
"""
cur.execute(query)

query = """
    create table Dim_Weather as (
        select 
            referencetime as time,
            trim(trailing ':0' from source_id) as weather_station,
            value as temperature
        from Weather_Gathered
    )
"""
cur.execute(query)

query = """
    alter table Dim_Weather
    add primary key(time, weather_station)
"""
cur.execute(query)

In [ ]:
query = """
    select 
        weather_station,
        count(*)
    from Dim_Weather
    group by weather_station
"""
pd.read_sql(query, conn).T

In [ ]:
query = """
    select *
    from Dim_Weather
    where time like '%2020-01%'
    limit 3
"""
pd.read_sql(query, conn)

In [ ]:
query = """
    drop table if exists Fact
"""
cur.execute(query)

query = """
    create table Fact 
    as(
        select 
            bel.time as time,
            date(bel.time) as day,
            bel.loc_id,
            s.nearest_station as weather_station
        from Dim_Belastning bel 
        left join
        (
            select 
                m.loc_id as loc_id,
                mns.nearest_station
            from 
                Dim_Municipality m,
                Municipality_Nearest_Station mns
            where m.municipality = mns.municipality
        ) s
        on bel.loc_id = s.loc_id  
    )
"""
cur.execute(query) 

In [ ]:
query = """
    select *
    from Fact
    where loc_id = 3
    limit 3
"""
pd.read_sql(query, conn)

In [ ]:
query = """
    select count(*)
    from Fact
"""
pd.read_sql(query, conn)

## Sample queries

i.  Find the locations with the highest and lowest average consumption and their municipality.

In [ ]:
query = """
    with a as (
        select
            f.loc_id as location_id,
            m.municipality as municipality,
            avg(b.consumption_kvah) as average_consumption
        from Fact f
        inner join Dim_Municipality m on f.loc_id=m.loc_id
        inner join Dim_Belastning b on (f.time=b.time and f.loc_id=b.loc_id)
        group by f.loc_id
    ), b as (
        select *
        from a
        order by average_consumption desc
        limit 1
    ), c as (
        select *
        from a
        order by average_consumption asc
        limit 1
    )
    select *
    from b 
    union all
    select *
    from c
"""
pd.read_sql(query, conn)

ii. For those two extremes, what was the mean temperature, oil price and el spot price at their day in 2019 of maximum consumption?

In [ ]:
query = """
    with temp_a as (
        select
            f.day as date,
            f.loc_id as location_id,
            f.weather_station as weather_station,
            avg(b.consumption_kvah) as average_consumption
            
        from Fact f
        inner join Dim_Belastning b on (f.time=b.time and f.loc_id=b.loc_id)
        where f.time like '%2019%'
        and f.loc_id in (10, 13)
        group by 
            location_id,
            date
            
    ), temp_b as (
        select
            location_id,
            max(average_consumption) as max_cons
        from temp_a
        group by location_id
        
    ), temp_c as (
        select 
            temp_a.date,
            temp_a.location_id,
            temp_a.weather_station,
            temp_a.average_consumption
        from
            temp_a,
            temp_b
            
        where temp_a.average_consumption = temp_b.max_cons
        
    )
        select 
            temp_c.date,
            temp_c.location_id,
            temp_c.average_consumption,
            avg(w.temperature) as avg_temp,
            o.price as oil_price,
            avg(e.price) as avg_el_price
        from
            temp_c
            inner join Dim_Elspot e on (date = date(e.time))
            inner join Dim_Oilspot o on (date = o.day)
            inner join Dim_Weather w on (date = date(w.time) and temp_c.weather_station = w.weather_station)
            
        group by date
"""
pd.read_sql(query, conn)

...And whats the mean daily temperature, oil price and el price in 2019 for those stations anyway?

In [ ]:
query = """
    select
        f.loc_id,
        avg(w.temperature) as avg_temp,
        avg(o.price) as avg_oil_price,
        avg(e.price) as avg_el_price
    from 
        Fact f
        inner join Dim_Weather w on (f.time=w.time and f.weather_id=w.weather_station)
        inner join Dim_Oilspot o on (f.day=o.day)
        inner join Dim_Elspot e on (f.time=e.time)
    
    where f.time like '%2019%'
    and f.loc_id in (10, 13)
    group by f.loc_id
    order by f.loc_id desc
"""
pd.read_sql(query, conn)

So the day of maximum el consumption were characterized by:
 * For 13: Temperature ~ten degrees C higher than the average. Oil price slightly lower than average. El price a bit higher than usual.
 * For 10: Temperature ~ten degrees C lower than the average. Oil price lower than average. El price quite higher than the usual.

### Join all interesting data to output table

In [ ]:
query = """
    select
        f.time as time,
        f.loc_id as location_id,
        b.consumption_kvah as el_consumption,
        w.temperature as degC,
        o.price as oil_price,
        e.price as el_price
        
    from Fact f
    left join Dim_Belastning b on (f.time=b.time and f.loc_id=b.loc_id)
    left join Dim_Weather w on (f.time=w.time and f.weather_id=w.weather_station) 
    left join Dim_Oilspot o on (f.day=o.day)
    left join Dim_Elspot e on (f.time=e.time)
"""
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{cwd}//sql_output//consumption_metatada_all_locations.csv')

In [ ]:
query = """
    select *
    from Dim_Weather
"""
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{cwd}//sql_output//Dim_Weather.csv')

In [ ]:
query = """
    select 
        f.time,
        f.loc_id,
        f.weather_station,
        b.consumption_kvah
    from Fact f
    inner join Dim_Belastning b on (f.time = b.time and f.loc_id = b.loc_id)
"""
df = pd.read_sql(query, conn)

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{cwd}//sql_output//Dim_Belastning_and_weather_station.csv')

In [ ]:
query = """
    select *
    from Dim_Oilspot
"""
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{cwd}//sql_output//Dim_Oilspot.csv')

In [ ]:
query = """
    select *
    from Dim_Elspot
"""
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
df.to_csv(f'{cwd}//sql_output//Dim_Elspot.csv')

In [ ]:
query = """
    select
        f.time as time,
        f.weather_id as weather_station,
        w.temperature as degC
        
    from Fact f
    right join Dim_Weather w on (f.time=w.time and f.weather_id=w.weather_station)
"""
df = pd.read_sql(query, conn)

In [ ]:
df.shape

In [ ]:
conn.close()
cur.close()